Student ID 25000148967

GitHub Repository Link https://github.com/robwhite-lis/Everything_Counts_Assessment_1

This is a Notebook about 2 of the UKs favourite topics, trains being late and the weather!

The dataset in question is available is the grandiosely named 

"Table 3138 - Train punctuality at recorded station stops by operator (periodic)"

and is available at https://dataportal.orr.gov.uk/statistics/performance/passenger-rail-performance/table-3138-train-punctuality-at-recorded-station-stops-by-operator-periodic/



In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests

df_url = 'https://raw.githubusercontent.com/robwhite-lis/Everything_Counts_Assessment_1/refs/heads/main/Train_punctuality.csv'
url_content = requests.get(df_url, verify = False).content
df_trains = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

In [5]:
df_trains.describe()

,Number of recorded station stops,MAT Number of recorded station stops
count,3.940000e+03,3.940000e+03
mean,6.474878e+05,8.026629e+06
std,1.394560e+06,1.753747e+07
min,0.000000e+00,6.600000e+01
25%,5.639950e+04,6.938212e+05
50%,2.288885e+05,2.574194e+06
75%,5.195705e+05,6.623621e+06
max,7.012797e+06,8.441222e+07


Train arrivals are monitored in a fairly strange way across periods, which roughly equate to months, but not quite, as there are 13 of them in a year. (They are actually 28 days each). Also the year runs from April to March!

In [9]:
df_trains.head()

,Time Period,Operator,Number of recorded station stops,Early,On Time within 59 seconds,Time to 3,Time to 5,Time to 10,Time to 15,Time to 20,...,On time within 59 seconds MAA,Time to 3 MAA,Time to 5 MAA,Time to 10 MAA,Time to 15 MAA,Time to 20 MAA,Time to 30 MAA,15 minutes plus MAA,20 minutes plus MAA,30 minutes plus MAA
0,Apr 2014 to Mar 2015 (Period 01),Great Britain,4736697,35.64,68.70,88.43,94.38,98.23,99.17,99.55,...,[z],[z],[z],[z],[z],[z],[z],[z],[z],[z]
1,Apr 2014 to Mar 2015 (Period 02),Great Britain,5061146,35.05,67.70,87.50,93.69,97.91,98.98,99.42,...,[z],[z],[z],[z],[z],[z],[z],[z],[z],[z]
2,Apr 2014 to Mar 2015 (Period 03),Great Britain,5084203,35.00,67.67,87.43,93.65,97.89,98.99,99.45,...,[z],[z],[z],[z],[z],[z],[z],[z],[z],[z]
3,Apr 2014 to Mar 2015 (Period 04),Great Britain,5123892,34.09,66.16,86.44,93.08,97.67,98.87,99.37,...,[z],[z],[z],[z],[z],[z],[z],[z],[z],[z]
4,Apr 2014 to Mar 2015 (Period 05),Great Britain,5131680,32.78,63.95,84.38,91.54,97.03,98.56,99.20,...,[z],[z],[z],[z],[z],[z],[z],[z],[z],[z]


OK so let's clean up the date period into something more useable

In [14]:
df_trains[["Start_Year", "Period"]] = df_trains["Time Period"].str.extract(r'(\d{4}).*Period (\d{2})')

In [15]:
df_trains["Start_Year"] = df_trains["Start_Year"].astype(int)
df_trains["Period"] = df_trains["Period"].astype(int)

In [24]:
df_trains[["Time Period", "Start_Year", "Period"]]

,Time Period,Start_Year,Period
0,Apr 2014 to Mar 2015 (Period 01),2014,1
1,Apr 2014 to Mar 2015 (Period 02),2014,2
2,Apr 2014 to Mar 2015 (Period 03),2014,3
3,Apr 2014 to Mar 2015 (Period 04),2014,4
4,Apr 2014 to Mar 2015 (Period 05),2014,5
...,...,...,...
3935,Apr 2025 to Mar 2026 (Period 03),2025,3
3936,Apr 2025 to Mar 2026 (Period 04),2025,4
3937,Apr 2025 to Mar 2026 (Period 05),2025,5
3938,Apr 2025 to Mar 2026 (Period 06),2025,6


In [28]:
df_trains[["Time Period", "Start_Year", "Period", "Season"]]

,Time Period,Start_Year,Period,Season
0,Apr 2014 to Mar 2015 (Period 01),2014,1,Spring
1,Apr 2014 to Mar 2015 (Period 02),2014,2,Spring
2,Apr 2014 to Mar 2015 (Period 03),2014,3,Spring
3,Apr 2014 to Mar 2015 (Period 04),2014,4,Summer
4,Apr 2014 to Mar 2015 (Period 05),2014,5,Summer
...,...,...,...,...
3935,Apr 2025 to Mar 2026 (Period 03),2025,3,Spring
3936,Apr 2025 to Mar 2026 (Period 04),2025,4,Summer
3937,Apr 2025 to Mar 2026 (Period 05),2025,5,Summer
3938,Apr 2025 to Mar 2026 (Period 06),2025,6,Summer


Now let's add a Season column

Bearing in mind that the periods run April to March, and there are 13 of them, I've made the following assingments which roughly translate as

Period 1 - 3 spring (April, May, June)
Period 4 - 6 Summe (July, August, September)r
Period 7 - 9 Autu (October, November, December)mn
Period 10 - 13  Win (January, February, March and a bit of April)ter


In [27]:
bins = [0,3,6,9,13]
labels = ["Spring", "Summer", "Autumn", "Winter"]
df_trains["Season"] = pd.cut(df_trains["Period"], bins=bins, labels=labels)